## Web Scrapping Practices

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns: 
  
  
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

In [19]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [98]:
def scrape_imdb(title_type, user_rating, start_date, end_date):
    
    base_url= "https://www.imdb.com"

    base_headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    search = f"/search/title/?title_type={title_type}&release_date={start_date}-01-01,{end_date}-12-31&user_rating={user_rating},10"

    base_response = requests.get(base_url+search, headers= base_headers)
    soup= BeautifulSoup(base_response.text)
    
    # Numbers
    numbers = [number.get_text().split(" ")[0] for number in soup.find_all("h3", class_="ipc-title__text")[:-1]]
    
    # Titles
    titles = [title.get_text() for title in soup.find_all("h3", class_="ipc-title__text")]
    titles.pop()
    
    # Years
    years_request = [item.get_text() for item in soup.find_all("span", class_ ="sc-b189961a-8 kLaxqf dli-title-metadata-item") ]
    years = [item for item in years_request if item.startswith('19')]
    
    # Ratings
    ratings_query = [rating.get_text() for rating in soup.find_all("span", class_= "ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating")]
    ratings = [item.split("\xa0")[0] for item in ratings_query]
    
    # Stars
    stars = [star.get_text() for star in soup.find_all("span", attrs={"class": "ipc-rating-star--voteCount"})]
    stars_split = [(star.split("(")[1])[:-1] for star in stars]
    
    # Movie Description
    movie_description = [description.get_text() for description in soup.find_all("div", class_= "ipc-html-content-inner-div")]
    
    # Movie time
    
    movie_time_split = [item.split("h") for item in years_request if item.endswith('m')]
    movie_time_minutes = [(int(time[0]))*60  +  int(time[1][:-1]) for time in movie_time_split]
    movie_time_minutes.append(98)
    
    
    #Data inside other links
    
    links = [l["href"] for l in soup.find_all("a", class_="ipc-title-link-wrapper")]
    
    genres = []
    directors = []
    actors = []

    for l in links:
            
            response = requests.get(base_url+l, headers= base_headers )
            soup = BeautifulSoup(response.text)
            
            # Genres
            genres.append([g.get_text() for g in soup.find_all("span", class_="ipc-chip__text") if g.get_text() != 'Back to top'] )
             
            # Directors
            directors.append([director.text.strip() for director in soup.find("a",class_= "ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link")]) 
        
            # Actors
            actors.append([a.get_text() for a in soup.find_all('a', attrs={"data-testid": "title-cast-item__actor"})])
            
    
    
    movies_df = pd.DataFrame({"number":numbers,"titles":titles,"year":years,"rating":ratings,"duration (min)": movie_time_minutes,"genre":genres,
                              "description":movie_description,"directors":directors,"actors":actors})
    movies_df.set_index("number", drop=True, inplace=True)
    
    
    return movies_df
    

# 🥳🥳🥳🥳🥳

In [99]:
movies_IMDB = scrape_imdb("feature", 7.5, 1990, 1992)

In [100]:
movies_IMDB

,titles,year,rating,duration (min),genre,description,directors,actors
number,,,,,,,,
1.,1. Das Schweigen der Lämmer,1991,8.6,118,"[Crime, Drama, Thriller]",A young F.B.I. cadet must receive the help of ...,[Jonathan Demme],"[Jodie Foster, Anthony Hopkins, Scott Glenn, T..."
2.,2. GoodFellas - Drei Jahrzehnte in der Mafia,1990,8.7,145,"[Biography, Crime, Drama]",The story of Henry Hill and his life in the ma...,[Martin Scorsese],"[Robert De Niro, Ray Liotta, Joe Pesci, Lorrai..."
3.,3. Terminator 2: Tag der Abrechnung,1991,8.6,137,"[Action, Adventure, Sci-Fi]","A cyborg, identical to the one who failed to k...",[James Cameron],"[Arnold Schwarzenegger, Linda Hamilton, Edward..."
4.,4. Mein Vetter Winnie,1992,7.6,99,"[Comedy, Crime]",Two New Yorkers accused of murder in rural Ala...,[Jonathan Lynn],"[Joe Pesci, Marisa Tomei, Ralph Macchio, Mitch..."
5.,5. Reservoir Dogs - Wilde Hunde,1992,8.3,130,"[Crime, Thriller]",When a simple jewelry heist goes horribly wron...,[Quentin Tarantino],"[Harvey Keitel, Tim Roth, Michael Madsen, Chri..."
6.,6. Grüne Tomaten,1991,7.7,162,[Drama],"Evelyn, an ordinary housewife, visits a nursin...",[Jon Avnet],"[Kathy Bates, Jessica Tandy, Mary Stuart Maste..."
7.,7. Der Pate 3,1990,7.6,84,"[Crime, Drama]","Follows Michael Corleone, now in his 60s, as h...",[Francis Ford Coppola],"[Al Pacino, Diane Keaton, Andy Garcia, Talia S..."
8.,8. Die Schöne und das Biest,1991,8.0,113,"[Animation, Family, Fantasy]",A prince cursed to spend his days as a hideous...,[Gary Trousdale],"[Paige O'Hara, Robby Benson, Jesse Corti, Rex ..."
9.,9. Total Recall - Die totale Erinnerung,1990,7.5,130,"[Action, Adventure, Sci-Fi]",When a man goes in to have virtual vacation me...,[Paul Verhoeven],"[Arnold Schwarzenegger, Sharon Stone, Michael ..."


In [102]:
movies_IMDB.to_csv("Movies_IMDB.csv", index = False)